In [1]:
# universally modules
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
sys.path.append("../src")

# preprocessing and transformation modules
import fasttext
import Preprocessing
from Features import buildFeatures
from Modelling import StackingModelling
from ModelSelection import ModelSelection, process_case
from Transformation import StackedTransformation, transformation

# Scikit-Learn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import TfidfVectorizer

# model algorithm
from sklearn.svm import LinearSVC, LinearSVR
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import SGDClassifier, SGDRegressor

# evaluation modules
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

/home/bigdata/anaconda3/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


### Parameters

In [11]:
# to speed up the process choose a sample size to randomly draw a sample of the whole daataset
sample_size = 10**100 

# remove all text that contain less than n chars
min_chars_per_text = 50

# which features will be used for the TF-IDF transformation
text_features = "text_preprocessed"

# General Preprocessing

This part is independent from the cases

### Initialization 

In [3]:
df = pd.read_csv("../resource/data/blogtext.csv")

# draw random sample for faster processing:
df = df.sample(sample_size)

### Filtering

In [4]:
# filter for a mininmal number of letters in a tweet:
df = df[df["text"].str.count(r"[a-zA-Z]") >= min_chars_per_text]
df = df.reset_index(drop=True)

### Feature Engineering

In [5]:
# append the data
features = [buildFeatures(text) for text  in tqdm(df["text"])]

# merge the features with the original dataset
df_preprocessed = df.merge(pd.DataFrame(features), left_index=True, right_index=True)

100%|██████████| 92/92 [00:00<00:00, 472.69it/s]


### Text Preprocessing

In [6]:
# use the preprocessing  module
preprocessor = Preprocessing.Preprocessing()
df_preprocessed["text_preprocessed"] = preprocessor.ProcessMany(df_preprocessed["text"])

# predict the main language
model = fasttext.load_model('../src/data/lid.176.ftz')
df_preprocessed["main_language"] = [model.predict(text)[0][0].split("__")[-1] for text in df_preprocessed["text_preprocessed"]]

# drop unnecassary features
df_filtered = df_preprocessed[(df_preprocessed["main_language"] == "en")]\
                .drop(["id","text","date","main_language"], axis= 1)

100%|██████████| 92/92 [00:03<00:00, 24.55it/s]


In [ ]:
# store df as pickle
df_filtered.to_pickle("./df_full_preprocessed.pkl")

# Individual Transformation

This part is dependent of the cases and need to be done individual for each case

### Clustering

In [7]:
def clustering():

    text_data_features = stacking.text_transformer.get_feature_names()
    text_data = stacking.X_train_text_transformed.toarray()

    df_text_cluster = pd.DataFrame(text_data, columns=text_data_features)

    # Die Features beschreiben die Worte im Text
    # Die Werte sind die TF*IDF-transformierten Textdaten

    numerical_data = stacking.X_train_numerical_transformed
    numerical_features = np.append(stacking.numerical_transformer.transformers_[0][1].get_feature_names(),\
                         stacking.X_train_numerical.columns.drop(categorial_variables))

    df_numerical_cluster = pd.DataFrame(numerical_data, columns=numerical_features)
    df_numerical_cluster


    # Die Features mit den x0 - xi Werten beschreiben die Ausprägungen die kategorialen Variablen
    # das jeweilige i beschreibt das i-te Element der im Punkt "Target Variable" definierten liste categorial_variables
    # Die verbleibenden Features (ohne xi) sind Standardskaliert, (x - \mu)/\sigma

    # THIS PART IS STILL MISSING

# Individual Modelling

This part is dependent of the cases and need to be done individual for each case

### Model Selection

In [8]:
# Split the data into  X and y
case_gender = {"target_type": "classification",
               "target_variable": "gender",
               "grid_search_metric": "accuracy",
               "text_features": "text_preprocessed",
               "categorial_variables": ["topic","sign"],
               "min_df_exponents" : [(1/4), (1/3), (1/2)],
               "n_gram_range" : [(1,1), (1,2), (2,2)],
               "use_tfidf": [True, False],
               "ml_algorithms_params": [(XGBClassifier,
                                   {'learning_rate': [0.1, 1, 1.5],
                                   'max_depth': [3, 6, 9],
                                   'n_estimators': [200, 600, 1200]}),
                                  
                                  (SGDClassifier,
                                   {"loss": ["hinge", "log", "modified_huber", "squared_hinge"],
                                   "penalty": ["l2", "l1", "elasticnet"],
                                   "alpha": [0.00001, 0.001]}),
                                  
                                  (LinearSVC,{"penalty": ["l1", "l2"],
                                   "loss": ["hinge", "squared_hinge"],
                                   "C": [0.8, 1, 1.2]})]
              }


case_topic = {"target_type": "classification",
               "target_variable": "topic",
               "grid_search_metric": "f1_weighted",
               "text_features": "text_preprocessed",
               "categorial_variables": ["gender","sign"],
               "min_df_exponents" : [(1/4), (1/3), (1/2)],
               "n_gram_range" : [(1,1), (1,2), (2,2)],
               "use_tfidf": [True, False],
               "ml_algorithms_params": [(XGBClassifier,
                                   {'learning_rate': [0.1, 1, 1.5],
                                   'max_depth': [3, 6, 9],
                                   'n_estimators': [200, 600, 1200]}),
                                  
                                  (SGDClassifier,
                                   {"loss": ["hinge", "log", "modified_huber", "squared_hinge"],
                                   "penalty": ["l2", "l1", "elasticnet"],
                                   "alpha": [0.01, 0.1, 1]}),
                                  
                                  (LinearSVC,{"penalty": ["l1", "l2"],
                                   "loss": ["hinge", "squared_hinge"],
                                   "C": [0.8, 1, 1.2]})]
              }



case_age = {"target_type": "regression",
               "target_variable": "age",
               "grid_search_metric": "neg_mean_squared_error",
               "text_features": "text_preprocessed",
               "categorial_variables": ["topic","gender","sign"],
               "min_df_exponents" : [(1/4), (1/3), (1/2)],
               "n_gram_range" : [(1,1), (1,2), (2,2)],
               "use_tfidf": [True, False],
               "ml_algorithms_params": [(XGBRegressor,
                                   {'learning_rate': [0.1, 1, 1.5],
                                   'max_depth': [3, 6, 9],
                                   'n_estimators': [200, 600, 1200]}),
                                  
                                  (SGDRegressor,
                                   {"loss": ["squared_loss", "huber"],
                                   "penalty": ["l2", "l1", "elasticnet"],
                                   "alpha": [0.01, 0.1, 1]}),
                                  
                                  (LinearSVR,
                                   {"loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
                                   "C": [0.8, 1, 1.2]})]
              }


case_topic = {"target_type": "classification",
               "target_variable": "sign",
               "grid_search_metric": "f1_weighted",
               "text_features": "text_preprocessed",
               "categorial_variables": ["gender","topic"],
               "min_df_exponents" : [(1/4), (1/3), (1/2)],
               "n_gram_range" : [(1,1), (1,2), (2,2)],
               "use_tfidf": [True, False],
               "ml_algorithms_params": [(XGBClassifier,
                                   {'learning_rate': [0.1, 1, 1.5],
                                   'max_depth': [3, 6, 9],
                                   'n_estimators': [200, 600, 1200]}),
                                  
                                  (SGDClassifier,
                                   {"loss": ["hinge", "log", "modified_huber", "squared_hinge"],
                                   "penalty": ["l2", "l1", "elasticnet"],
                                   "alpha": [0.01, 0.1, 1]}),
                                  
                                  (LinearSVC,{"penalty": ["l1", "l2"],
                                   "loss": ["hinge", "squared_hinge"],
                                   "C": [0.8, 1, 1.2]})]
              }


In [10]:
for case in [case_gender, case_topic, case_age, case_topic]:
    df_results = process_case(transformation, case, df_filtered)
    df_results.to_pickle(f'./pd_df_cv_{case["target_variable"]}_{str(pd.Timestamp.now())}.pkl')

  0%|          | 0/54 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s


Processing target variable {version["target_variable"]}
Starting Grid search for numerical model ...
Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done  66 out of  81 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s


Starting Grid search for text model ...
Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=-1)]: Done  81 out of  81 | elapsed:    0.4s finished
  2%|▏         | 1/54 [00:00<00:45,  1.16it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.0s


...

Processing target variable {version["target_variable"]}
Starting Grid search for numerical model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  93 out of 108 | elapsed:    0.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s


Starting Grid search for text model ...
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  93 out of 108 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.2s finished
 98%|█████████▊| 53/54 [00:47<00:00,  1.33it/s][Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:    0.1s finished
100%|██████████| 54/54 [00:47<00:00,  1.14it/s]


Processing target variable {version["target_variable"]}
Starting Grid search for numerical model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
Starting Grid search for text model ...
Fitting 3 folds for each of 12 candidates, totalling 36 fits
